In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
def get_league_info(current_id):
    """Returns the league information from the sleeper api.

    Args:
        current_id (string or integer): A string/integer containing the most recent league id.

    Returns:
        dict: The league's current information.
    """

    league_url = f"https://api.sleeper.app/v1/league/{current_id}"
    response = requests.get(league_url)
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: {response.status_code}")

    return(data)

def get_all_league_info(league_id):
    """Generates a list of all league information from the sleeper api, calls the get_league_id function.

    Args:
        league_id (string or integer): A string/integer containing the most recent league id.

    Returns:
        dict: A dictionary containing all previous league info: id, season, playoff_week_start
    """

    while league_id != None:
        league_data = get_league_info(league_id)
        try:
            league_info["league_id"].append(league_id)
            league_info["season"].append(league_data["season"])
            league_info["playoff_week_start"].append(league_data["settings"]["playoff_week_start"])

            league_id = league_data["previous_league_id"]

        except NameError:
            # create league dictionary if it doesn't exist
            league_info = {"league_id": [],
                           "season": [],
                           "playoff_week_start":[]}
            
            

            league_info["league_id"].append(league_id)
            league_info["season"].append(league_data["season"])
            league_info["playoff_week_start"].append(league_data["settings"]["playoff_week_start"])

            league_id = league_data["previous_league_id"]


    
    return(pd.DataFrame(league_info))

In [36]:
def get_roster_id(league_id, year):
    users_dict = {}
    rosters_dict = {}

    users_url = f"https://api.sleeper.app/v1/league/{league_id}/users"
    rosters_url = f"https://api.sleeper.app/v1/league/{league_id}/rosters"

    # Make the API request
    users_response = requests.get(users_url)
    # Check if the request was successful (status code 200)
    if users_response.status_code == 200:
        # Parse the JSON response
        users = users_response.json()
    else:
        print(f"Error: {users_response.status_code}")

    # Make the API request
    rosters_response = requests.get(rosters_url)
    # Check if the request was successful (status code 200)
    if rosters_response.status_code == 200:
        # Parse the JSON response
        rosters = rosters_response.json()
    else:
        print(f"Error: {rosters_response.status_code}")

    for i, (user, roster) in enumerate(zip(users, rosters)):
        users_dict[i+1] = [user["display_name"], user["user_id"], year]
        rosters_dict[i+1] = [roster['owner_id'], roster['roster_id']]

    users_df = pd.DataFrame.from_dict(users_dict, orient = 'index', columns=['display_name', 'user_id', "Season"])
    rosters_df = pd.DataFrame.from_dict(rosters_dict, orient = 'index', columns=['user_id', 'roster_id'])

    merged_df = pd.merge(users_df, rosters_df, on = 'user_id')

    return(merged_df)

In [4]:
def get_matchups(league_id, week_number):
    api_url = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week_number}"
    # Make the API request
    response = requests.get(api_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        matchups = response.json()
        return(matchups)
    else:
        print(f"Error: {response.status_code}")

In [6]:
def get_matchups_season(league_id, season, play_off_week_start, all_rosters):
    users_df = all_rosters[all_rosters["Season"] == str(season)]
    season_matchups = pd.DataFrame()
    for i in range(18):
        week = i+1
        matchups_response = get_matchups(league_id, week_number = week)
        
        if matchups_response != []:
            matchups = {"points": [],
                        "roster_id": [],
                        "matchup_id": []}
            
            for matchup in matchups_response:
                matchups["points"].append(matchup["points"])
                matchups["roster_id"].append(matchup["roster_id"])
                matchups["matchup_id"].append(matchup["matchup_id"])

            
            matchups_df = pd.DataFrame(matchups)
            matchups_df = matchups_df.sort_values(by=['matchup_id', 'points']).reset_index(drop=True)
            matchups_df['pair'] = matchups_df.groupby('matchup_id').cumcount() + 1
            matchups_df = matchups_df.merge(users_df, on = 'roster_id')
            matchups_df = matchups_df[matchups_df["matchup_id"].notnull()] # if it's playoffs, teams that aren't included still return a matchup but shouldn't be kept

            matchups_df_wide = matchups_df.pivot(index='matchup_id', columns='pair', values=['roster_id', 'points', 'display_name', 'user_id'])
            matchups_df_wide.columns = [f"{col[0]}_{col[1]}" for col in matchups_df_wide.columns]
            matchups_df_wide["season"] = season
            matchups_df_wide["week"] = week
            matchups_df_wide["match_type"] = matchups_df_wide['week'].apply(lambda x: 'Regular Season' if x < int(play_off_week_start) else 'Playoffs')

            season_matchups = pd.concat([season_matchups, matchups_df_wide])  

    season_matchups = season_matchups[(season_matchups["points_1"] > 0) & (season_matchups["points_2"] > 0)]
    season_matchups['user_pair'] = season_matchups.apply(lambda row: tuple(sorted([row['display_name_1'], row['display_name_2']])), axis=1)
    season_matchups["Winner"] = season_matchups.apply(lambda row: row['display_name_1'] if row['points_1'] > row['points_2'] else row['display_name_2'], axis=1)
    season_matchups["Loser"] = season_matchups.apply(lambda row: row['display_name_2'] if row['points_1'] > row['points_2'] else row['display_name_1'], axis=1)

    return(season_matchups)


# Starting from scratch here is the order of operations to get the info

## 1. here we get the league info for each season:

In [7]:
league_info = get_all_league_info(1073659471932538880) # dataframe with league_id, season, playoff_week_start
league_info

,league_id,season,playoff_week_start
0,1073659471932538880,2024,15
1,936029369854734336,2023,15
2,851216576215617536,2022,15
3,650041395410722816,2021,15
4,515606348271226880,2020,14
5,456679182605217792,2019,14


## 2. here we get the rosters for each season
- this helps account for changing users
- roster_id is also not constant year by year so we need to make sure we are merging the right user for each matchup

In [37]:
all_rosters = pd.DataFrame()

for season_id, year in zip(league_info.league_id, league_info.season):
    year_roster = get_roster_id(season_id, year)

    all_rosters = pd.concat([all_rosters, year_roster])

all_rosters.head()
    

,display_name,user_id,Season,roster_id
0,BLoesch,456677174343102464,2024,1
1,jloesch,456846148506742784,2024,2
2,Sousdini,456865634018390016,2024,3
3,charifelm,456881030087831552,2024,5
4,awilley19,456939547255631872,2024,4


## 3. here we get the matchups for a season

In [9]:
season_matchups = get_matchups_season(league_info["league_id"][0], league_info["season"][0], league_info["playoff_week_start"][0], all_rosters)

season_matchups.head()

,roster_id_1,roster_id_2,points_1,points_2,display_name_1,display_name_2,user_id_1,user_id_2,season,week,match_type,user_pair,Winner,Loser
matchup_id,,,,,,,,,,,,,,
1,5,3,115.96,145.46,awilley19,Sousdini,456939547255631872,456865634018390016,2024,1,Regular Season,"(Sousdini, awilley19)",Sousdini,awilley19
2,8,10,147.76,150.76,dompappa,Aiirik91,459487653503430656,757636343194050560,2024,1,Regular Season,"(Aiirik91, dompappa)",Aiirik91,dompappa
3,4,7,123.86,124.8,charifelm,TheGoldenClutch,456881030087831552,458263550834503680,2024,1,Regular Season,"(TheGoldenClutch, charifelm)",TheGoldenClutch,charifelm
4,2,9,143.36,144.98,jloesch,geocesar96,456846148506742784,460258882422304768,2024,1,Regular Season,"(geocesar96, jloesch)",geocesar96,jloesch
5,1,6,119.3,150.1,BLoesch,APTwoOne,456677174343102464,456971412364587008,2024,1,Regular Season,"(APTwoOne, BLoesch)",APTwoOne,BLoesch


## 4. Finally get the matchups for every possible season
- one thing I should add is checking if the week started
    - this would mostly be relevant for the current season
    - we only want to include completed games

In [293]:
for index, row in league_info.iterrows():
    print(row["league_id"], row["season"], row["playoff_week_start"])

1073659471932538880 2024 15
936029369854734336 2023 15
851216576215617536 2022 15
650041395410722816 2021 15
515606348271226880 2020 14
456679182605217792 2019 14


In [298]:
all_matchups = pd.DataFrame()

for id, season, playoff_start in zip(league_info.league_id, league_info.season, league_info.playoff_week_start):
    season_matchups = get_matchups_season(id, season, playoff_start, all_rosters)

    all_matchups = pd.concat([all_matchups, season_matchups])
    

C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_25400\3343561248.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  season_matchups = pd.concat([season_matchups, matchups_df_wide])
C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_25400\3343561248.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  season_matchups = pd.concat([season_matchups, matchups_df_wide])
C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_25400\3343561248.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, t

In [300]:
all_matchups.to_csv("test_matchups_new.csv")

In [308]:
all_matchups[all_matchups["user_pair"] == ("BLoesch", "jloesch")].sort_values(by = "season")

,roster_id_1,roster_id_2,points_1,points_2,display_name_1,display_name_2,user_id_1,user_id_2,season,week,...,Winner,Loser,roster_id_1.0,roster_id_2.0,points_1.0,points_2.0,display_name_1.0,display_name_2.0,user_id_1.0,user_id_2.0
matchup_id,,,,,,,,,,,,,,,,,,,,,
5.0,1,2,141.56,184.32,BLoesch,jloesch,456677174343102464,456846148506742784,2019,2,...,jloesch,BLoesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,2,1,121.1,125.08,jloesch,BLoesch,456846148506742784,456677174343102464,2019,11,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,1,2,138.66,138.78,BLoesch,jloesch,456677174343102464,456846148506742784,2020,4,...,jloesch,BLoesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,2,1,106.6,168.46,jloesch,BLoesch,456846148506742784,456677174343102464,2020,13,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,2,1,134.66,172.0,jloesch,BLoesch,456846148506742784,456677174343102464,2021,1,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,2,1,112.56,149.92,jloesch,BLoesch,456846148506742784,456677174343102464,2021,10,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,2,1,169.0,183.16,jloesch,BLoesch,456846148506742784,456677174343102464,2021,16,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,2,1,125.1,146.46,jloesch,BLoesch,456846148506742784,456677174343102464,2022,4,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,2,1,152.94,178.2,jloesch,BLoesch,456846148506742784,456677174343102464,2022,13,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#testing

In [10]:
league_id = 1073659471932538880
season = 2024
play_off_week_start = 15

users_df = all_rosters[all_rosters["Season"] == str(season)]
season_matchups = pd.DataFrame()
for i in range(3):
    week = i+1
    matchups_response = get_matchups(league_id, week_number = week)
    
    if matchups_response != []:
        matchups = {"points": [],
                    "roster_id": [],
                    "matchup_id": []}
        
        for matchup in matchups_response:
            matchups["points"].append(matchup["points"])
            matchups["roster_id"].append(matchup["roster_id"])
            matchups["matchup_id"].append(matchup["matchup_id"])

        
        matchups_df = pd.DataFrame(matchups)
        matchups_df = matchups_df.sort_values(by=['matchup_id', 'points']).reset_index(drop=True)
        matchups_df['pair'] = matchups_df.groupby('matchup_id').cumcount() + 1
        matchups_df = matchups_df.merge(users_df, on = 'roster_id')
        matchups_df = matchups_df[matchups_df["matchup_id"].notnull()] # if it's playoffs, teams that aren't included still return a matchup but shouldn't be kept

        matchups_df_wide = matchups_df.pivot(index='matchup_id', columns='pair', values=['roster_id', 'points', 'display_name', 'user_id'])
        matchups_df_wide.columns = [f"{col[0]}_{col[1]}" for col in matchups_df_wide.columns]
        matchups_df_wide["season"] = season
        matchups_df_wide["week"] = week
        matchups_df_wide["match_type"] = matchups_df_wide['week'].apply(lambda x: 'Regular Season' if x < int(play_off_week_start) else 'Playoffs')

        season_matchups = pd.concat([season_matchups, matchups_df_wide])  

season_matchups = season_matchups[(season_matchups["points_1"] > 0) & (season_matchups["points_2"] > 0)]
season_matchups['user_pair'] = season_matchups.apply(lambda row: tuple(sorted([row['display_name_1'], row['display_name_2']])), axis=1)
season_matchups["Winner"] = season_matchups.apply(lambda row: row['display_name_1'] if row['points_1'] > row['points_2'] else row['display_name_2'], axis=1)
season_matchups["Loser"] = season_matchups.apply(lambda row: row['display_name_2'] if row['points_1'] > row['points_2'] else row['display_name_1'], axis=1)

In [13]:
matchups_response = get_matchups(1073659471932538880, 1)

matchups = {"points": [],
            "roster_id": [],
            "matchup_id": []}

for matchup in matchups_response:
    matchups["points"].append(matchup["points"])
    matchups["roster_id"].append(matchup["roster_id"])
    matchups["matchup_id"].append(matchup["matchup_id"])

matchups


{'points': [119.3,
  143.36,
  145.46,
  123.86,
  115.96,
  150.1,
  124.8,
  147.76,
  144.98,
  150.76],
 'roster_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'matchup_id': [5, 4, 1, 3, 1, 5, 3, 2, 4, 2]}

In [14]:
users_df

,display_name,user_id,roster_id,Season
1,BLoesch,456677174343102464,1,2024
2,jloesch,456846148506742784,2,2024
3,Sousdini,456865634018390016,3,2024
4,charifelm,456881030087831552,4,2024
5,awilley19,456939547255631872,5,2024
6,APTwoOne,456971412364587008,6,2024
7,TheGoldenClutch,458263550834503680,7,2024
8,dompappa,459487653503430656,8,2024
9,geocesar96,460258882422304768,9,2024
10,Aiirik91,757636343194050560,10,2024


In [34]:
league_id = 1073659471932538880
year = 2024

users_dict = {}
rosters_dict = {}

users_url = f"https://api.sleeper.app/v1/league/{league_id}/users"
rosters_url = f"https://api.sleeper.app/v1/league/{league_id}/rosters"

# Make the API request
users_response = requests.get(users_url)
# Check if the request was successful (status code 200)
if users_response.status_code == 200:
    # Parse the JSON response
    users = users_response.json()
else:
    print(f"Error: {users_response.status_code}")

# Make the API request
rosters_response = requests.get(rosters_url)
# Check if the request was successful (status code 200)
if rosters_response.status_code == 200:
    # Parse the JSON response
    rosters = rosters_response.json()
else:
    print(f"Error: {rosters_response.status_code}")

for i, (user, roster) in enumerate(zip(users, rosters)):
    users_dict[i+1] = [user["display_name"], user["user_id"], year]
    rosters_dict[i+1] = [roster['owner_id'], roster['roster_id']]

users_df = pd.DataFrame.from_dict(users_dict, orient = 'index', columns=['display_name', 'user_id', "Season"])
rosters_df = pd.DataFrame.from_dict(rosters_dict, orient = 'index', columns=['user_id', 'roster_id'])

merged_df = pd.merge(users_df, rosters_df, on = 'user_id')



In [18]:
for user in users:
    print(user['display_name'], user["user_id"])

BLoesch 456677174343102464
jloesch 456846148506742784
Sousdini 456865634018390016
charifelm 456881030087831552
awilley19 456939547255631872
APTwoOne 456971412364587008
TheGoldenClutch 458263550834503680
dompappa 459487653503430656
geocesar96 460258882422304768
Aiirik91 757636343194050560


In [19]:
for roster in rosters:
    print(roster["roster_id"])

1
2
3
4
5
6
7
8
9
10


In [21]:
rosters[0].keys()

dict_keys(['co_owners', 'keepers', 'league_id', 'metadata', 'owner_id', 'player_map', 'players', 'reserve', 'roster_id', 'settings', 'starters', 'taxi'])

In [22]:
rosters[0]['owner_id']

'456677174343102464'

In [30]:
users_df

,display_name,user_id,Season
1,BLoesch,456677174343102464,2024
2,jloesch,456846148506742784,2024
3,Sousdini,456865634018390016,2024
4,charifelm,456881030087831552,2024
5,awilley19,456939547255631872,2024
6,APTwoOne,456971412364587008,2024
7,TheGoldenClutch,458263550834503680,2024
8,dompappa,459487653503430656,2024
9,geocesar96,460258882422304768,2024
10,Aiirik91,757636343194050560,2024


In [31]:
rosters_df

,owner_id,roster_id,Season
1,456677174343102464,1,2024
2,456846148506742784,2,2024
3,456865634018390016,3,2024
4,456939547255631872,4,2024
5,456881030087831552,5,2024
6,456971412364587008,6,2024
7,458263550834503680,7,2024
8,757636343194050560,8,2024
9,459487653503430656,9,2024
10,460258882422304768,10,2024


In [35]:
merged_df

,display_name,user_id,Season,roster_id
0,BLoesch,456677174343102464,2024,1
1,jloesch,456846148506742784,2024,2
2,Sousdini,456865634018390016,2024,3
3,charifelm,456881030087831552,2024,5
4,awilley19,456939547255631872,2024,4
5,APTwoOne,456971412364587008,2024,6
6,TheGoldenClutch,458263550834503680,2024,7
7,dompappa,459487653503430656,2024,9
8,geocesar96,460258882422304768,2024,10
9,Aiirik91,757636343194050560,2024,8
